In [1]:
### 2021 Census Data (ACS 1-year)for Selected Variables - Baltimore City



In [1]:
# Dependencies
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import requests
import censusdata 
#import censusgeocode as cg

#from scipy.stats import linregress


In [3]:
# Plotly is a library for creating interactive, high-quality visualizations. 

# Seaborn and matplotlib are better for static images

# See https://medium.com/codex/say-goodbye-to-matplotlib-and-seaborn-for-your-python-plots-5cade2f7fba2
#import plotly.graph_objects as go
#import plotly.express as px

In [4]:
#import os

In [5]:
#os.getcwd()

In [6]:
pd.set_option('display.max_rows', 200)

In [7]:
pd.set_option('max_colwidth', 250)

In [3]:
# American Community Survey 1-Year Data 

data = censusdata.download('acs1', 2016,
                          censusdata.censusgeo([('state', '24'),
                                         ('county', '510'),
                                          ('tract', '*')]),
                                        ["B01001_001E", 
                                          "B02001_002E",
                                          "B02001_003E",
                                          "B02001_005E",              
                                          "B02001_008E",
                                          "B03001_003E",
                                          "B05002_013E",
                                          "B08301_001E",               
                                          "B08301_010E",
                                          "B15003_001E",
                                          "B15003_016E",               
                                          "B15003_017E",
                                          "B15003_021E",               
                                          "B15003_022E", 
                                          ])
                          
census_df = pd.DataFrame(data)
census_df.head()

ValueError: Unexpected response (URL: https://api.census.gov/data/2016/acs/acs1?get=NAME,B01001_001E,B02001_002E,B02001_003E,B02001_005E,B02001_008E,B03001_003E,B05002_013E,B08301_001E,B08301_010E,B15003_001E,B15003_016E,B15003_017E,B15003_021E,B15003_022E&for=tract:*&in=state:24+county:510): error: unknown/unsupported geography heirarchy 

In [ ]:
census_df = census_df.reset_index()

In [ ]:
census_df = census_df.rename(columns={"index": "Geography"})
census_df.head()

In [ ]:
census_df.shape

In [ ]:
census_df.dtypes

In [ ]:
# remove commas from the text in "Geography" column so that I can complete the following step of removing extraneous text
census_df["Geography"]= census_df["Geography"].astype(str).str.replace(","," ")
census_df

In [ ]:
# this didn't work until I did the step above of removing the commas from the text in the "Geography" column

# Slice to a specific ending point from the end of the string: You can use the str.slice(stop) method to slice a string column up to a specific stop index from the end of the string. 
# see https://medium.com/geekculture/how-do-you-use-slice-method-in-the-pandas-dataframe-on-string-data-type-columns-6a8fd02c15eb
# code below "slices"/ removes text from the 'Geography' column up to the 6th character from the END of the string
census_df["Geography"]= census_df["Geography"].str.slice(-6)
census_df

In [ ]:
# add census state code (24) and city code (510) to each value in the "Geography column" by concatenating 24510 with the tract #
# this way, we can more easily merge with dataframes from other census dataframes for commercial corridors
census_df["Geography"] = "24510" + census_df["Geography"]
census_df.head()

In [ ]:
census_df.columns

In [ ]:
# Create Geographic Identifier ("GEOID") for each census tract by adding state fips code + county fips code + census tract code
# see https://www.census.gov/programs-surveys/geography/guidance/geo-identifiers.html#:~:text=The%20full%20GEOID%20for%20many,codes%2C%20in%20which%20they%20nest.
#census_df["GEOID"] = census_df['state'] + census_df['county'] + census_df['tract']
#census_df

In [ ]:
# Replace the census variable codes (such as "B19013_001E") in the dataframe with text so it's understandable
census_df = census_df.rename(columns={"B19001_001E": "Total households",
                                      "B01001_001E": "Population",
                                      "tract": "Census Tract",  
                                      "B08301_001E": "Workers_16_yrs_and_over",  
                                      "B08301_010E": "Commute_to_work_public_transportation",       
                                      
                                      "B02001_002E": "Pop. white",
                                      "B02001_003E": "Pop. Black",
                                      "B02001_005E": "Pop. Asian",        
                                      "B02001_008E": "Pop. two or more races",
                                      "B03001_003E": "Pop. Hispanic origin",
                                      "B05002_013E": "# Foreign-born",
                                      "B11002_001E": "Household population",
                                      "B15003_001E": "Population_25_yrs_and_over",
                                      "B15003_016E": "# persons 12th grade, no diploma",
                                      "B15003_017E": "# persons graduated high school",
                                      "B15003_021E": "# persons Associate's degree",
                                      "B15003_022E": "# persons Bachelor's degree"
                                      
                                     })       
census_df.head()                              



In [ ]:
# merge the census_pd dataframe with the age_final dataframe on the common column "GEOID"
#census_joined2 = pd.concat([census_pd, age_final], axis=1)
#census_joined2

In [ ]:
# GIS Analyst, Patrick, provided a csv (created from ARcGIS) that contains a key to match Baltimore's commercial corridors 
# with specific GEOIDS 

# Store filepath in a variable
corridor_key = "raw_data/corr_key.csv"

# Read the file with the pandas library
corr_key_df2 = pd.read_csv(corridor_key)
corr_key_df2.dtypes

In [ ]:
# Change data type of the "GEOID" column so that it's a string/ text and not an integer
corr_key_df2["GEOID"] = corr_key_df2["GEOID"].astype('str')
corr_key_df2.dtypes

In [ ]:
corr_key_df2

In [ ]:
corr_key_df2.columns

In [ ]:
#corr_key_df = corr_key_df2.sort_values("GEOID")
#corr_key_df

In [ ]:
# print all of the values in the "GEOID" column
#geoid_list = census_joined2['GEOID'].tolist()
#print(geoid_list)

In [ ]:
# rename "GEOID" column so can merge it with census_df on common column name "Geography"
corr_key_df2 = corr_key_df2.rename(columns={"GEOID": "Geography"
                                           })
corr_key_df2

In [ ]:
# combine the census_df dataframe with the corr_key_df2 dataframe 

corridors_df = pd.merge(
    census_df, corr_key_df2, on="Geography")
corridors_df

In [ ]:
# remove any rows with NaN ("Not a Number") - used for missing values, by using .dropna() bc we only want rows that match to corridors we're studying
corridors_df = corridors_df.dropna()
corridors_df

In [ ]:
corridors_df.columns

In [ ]:
# Use .groupby and .agg to sum the amounts by corridor
corridors_sum = corridors_df.groupby(['Corridor'], as_index=False).agg(
    {"Population": sum,
     "Pop. white": sum,
     "Pop. Black": sum,
     "Pop. Asian": sum,
     "Pop. two or more races": sum,
     "Pop. Hispanic origin": sum,
     "# Foreign-born": sum,
     "Workers_16_yrs_and_over": sum,
     "Commute_to_work_public_transportation": sum,
     "Population_25_yrs_and_over": sum,
     "# persons 12th grade, no diploma": sum,
     "# persons graduated high school": sum,
     "# persons Associate's degree": sum,
     "# persons Bachelor's degree": sum
    }) 
corridors_sum

In [ ]:
corridors_sum["Percent Black"] = 100 * \
    corridors_sum["Pop. Black"].astype(
        int) / corridors_sum["Population"].astype(
        int) 

corridors_sum["Percent Black"] = corridors_sum["Percent Black"].astype(float).round(1)

In [ ]:
corridors_sum["Percent Hispanic origin"] = 100 * \
    corridors_sum["Pop. Hispanic origin"].astype(
        int) / corridors_sum["Population"].astype(
        int) 

corridors_sum["Percent Hispanic origin"] = corridors_sum["Percent Hispanic origin"].astype(float).round(1)

In [ ]:
corridors_sum["Percent White"] = 100 * \
    corridors_sum["Pop. white"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent White"] = corridors_sum["Percent White"].astype(float).round(1)


In [ ]:
corridors_sum["Percent two or more races"] = 100 * \
    corridors_sum["Pop. two or more races"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent two or more races"] = corridors_sum["Percent two or more races"].astype(float).round(1)


In [ ]:
corridors_sum["Percent 12th grade, no diploma"] = 100 * \
    corridors_sum["# persons 12th grade, no diploma"].astype(
        int) / corridors_sum["Population_25_yrs_and_over"].astype(
        int) 
corridors_sum["Percent 12th grade, no diploma"] = corridors_sum["Percent 12th grade, no diploma"].astype(float).round(1)


In [ ]:
corridors_sum["Percent graduated high school"] = 100 * \
    corridors_sum["# persons graduated high school"].astype(
        int) / corridors_sum["Population_25_yrs_and_over"].astype(
        int) 
corridors_sum["Percent graduated high school"] = corridors_sum["Percent graduated high school"].astype(float).round(1)


In [ ]:
corridors_sum["Percent earned Associate's degree"] = 100 * \
    corridors_sum["# persons Associate's degree"].astype(
        int) / corridors_sum["Population_25_yrs_and_over"].astype(
        int) 
corridors_sum["Percent earned Associate's degree"] = corridors_sum["Percent earned Associate's degree"].astype(float).round(1)


In [ ]:
corridors_sum["Percent earned Bachelor's degree"] = 100 * \
    corridors_sum["# persons Bachelor's degree"].astype(
        int) / corridors_sum["Population_25_yrs_and_over"].astype(
        int) 
corridors_sum["Percent earned Bachelor's degree"] = corridors_sum["Percent earned Bachelor's degree"].astype(float).round(1)


In [ ]:
corridors_sum["Percent Foreign-born"] = 100 * \
    corridors_sum["# Foreign-born"].astype(
        int) / corridors_sum["Population"].astype(
        int) 
corridors_sum["Percent Foreign-born"] = corridors_sum["Percent Foreign-born"].astype(float).round(1)
corridors_sum

In [ ]:
corridors2021 = corridors_sum.copy()
corridors2021

In [ ]:
# see https://medium.com/codex/say-goodbye-to-matplotlib-and-seaborn-for-your-python-plots-5cade2f7fba2
# Use plotly to create scatter plot
fig = px.scatter(corridors2021, x="Percent Black", y= "Percent earned Bachelor's degree",
                hover_name = "Corridor", color="Corridor")
fig.update_layout({
'plot_bgcolor':'rgba(0,0,0,0)',
'paper_bgcolor':'rgba(0,0,0,0)',
'title': "College Degree"
})
fig.show()

In [ ]:
# create bar chart using plotly
Black = corridors2021.groupby(['Corridor'])['Percent Black'].sum().reset_index()
Black.columns = ['Corridor','Percent Black']
top_Black = Black.sort_values(by = ['Percent Black'],ascending = False)[:10]


fig = px.bar(top_Black, x = 'Corridor', y="Percent Black")
fig.update_layout({
'plot_bgcolor': 'rgba(0, 0, 0, 0)',
'paper_bgcolor': 'rgba(0, 0, 0, 0)',
'title': 'Corridors with highest percentage Black residents'})

fig.show()

In [ ]:
# how to use Dash to share your plotly visualizations: https://towardsdatascience.com/dash-for-beginners-create-interactive-python-dashboards-338bfcb6ffa4

In [ ]:
# Export df as an Excel file
#corridors_2021_final.to_excel("CommCorr_demographics_2021.xlsx", index = False)

In [ ]:
# import csv file of Baltimore City (as a whole) demographics
# first save the data from the csv in variable, "baltcity_2021" 

#baltcity_2021 = "BaltCity_demographics_2021.csv"

In [ ]:
# Create dataframes by reading the variable data from the code block above
#balt_df2 = pd.read_csv(baltcity_2021)
#balt_df2

In [ ]:
#balt_df = balt_df2.reset_index(drop=True)

In [ ]:
# merge the dataframes on the common column "Corridor"
#corridors_balt_merge = pd.concat([corridors_2021_final, balt_df], axis=0)
#corridors_balt_merge = corridors_balt_merge.reset_index(drop=True)
#corridors_balt_merge.columns

In [ ]:
# import csv file of Maryland demographics
# first save the data from the csv in variable, "MD_2021" 

#MD_2021 = "MD_demographics_2021.csv"

In [ ]:
# Create dataframe by reading the variable data from the code block above
#MD_df = pd.read_csv(MD_2021)
#MD_df

In [ ]:
#MD_df = MD_df.reset_index(drop=True)

In [ ]:
# merge the dataframes on the common column "Corridor"
#Demog = pd.concat([corridors_balt_merge, MD_df], axis=0)
#Demog = Demog.reset_index(drop=True)
#Demog

In [ ]:
# Change names of corridors (replace string using apply() function with lambda)
#Demographics_Master = Demog.apply(lambda x: x.replace({"Baltimore city, Maryland":"Baltimore City", "Penn Ave": "Pennsylvania Avenue",
#                                                       "Belair Rd": "Belair Road", "North Ave": "North Avenue"}, regex=True))
#Demographics_Master                   

In [ ]:
#Demographics_Master.to_excel("CommCorr_Tableau-2021/CommCorr_Tableau_2021rev.xlsx")